In [1]:
import traceback
from dotenv import load_dotenv
import pandas as pd
import duckdb
import pyarrow.parquet as pq
# from azure.ai.textanalytics import TextAnalyticsClient, HealthcareEntityRelation
# from azure.core.credentials import AzureKeyCredential

# Set up environment variables
load_dotenv()

# Set flags
DEBUG = False

data_dir = %env DATA_DIR
database_url = %env DATABASE_URL
azure_key = %env AZURE_KEY
azure_endpoint = %env AZURE_ENDPOINT

# Load SQL extension
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False


# DuckDB loaded using $DATABASE_URL from `.env`
con = duckdb.connect(f'{data_dir}/cerulean.duckdb')

%sql PRAGMA enable_progress_bar=true;
%sql db_list << PRAGMA database_list;
display(db_list)

# Load data from parquet
# Load schema metadata from ${data_dir}physionet_schema.csv
tables = pd.read_csv(f'{data_dir}/physionet_schema.csv', delimiter='\t', 
                usecols=['schema', 'table']).drop_duplicates().reset_index(drop=True)

# Filter to mimiciv_note schema for now
tables = tables[tables['schema'] == 'mimiciv_note']

,seq,name,file
0,4,cerulean,/Users/christopher/cerulean-data/cerulean.duckdb


In [2]:
# Load data from parquet using pandas
for (schema, table) in tables.values:
    try:
        print(f'1 of 3: Reading {schema}.{table} from parquet')
        df = pd.read_parquet(f'{data_dir}/parquet/{schema}/{table}.parquet')
        # table_data = pq.read_table(f'{data_dir}/parquet/{schema}/{table}.parquet')
        # df = table_data.to_pandas()
        # load df into duckdb
        %sql DROP TABLE IF EXISTS {{schema}}.{{table}}
        %sql CREATE SCHEMA IF NOT EXISTS {{schema}}
        # con.execute(f"DROP TABLE IF EXISTS {schema}.{table}")
        # con.execute(f"CREATE SCHEMA IF NOT EXISTS {schema}")
        print(f'2 of 3: Loading {schema}.{table} into duckdb')
        %sql CREATE TABLE {{schema}}.{{table}} AS SELECT * FROM df
        # con.execute(f"CREATE TABLE {schema}.{table} AS SELECT * FROM table_data")
        # con.commit()
        print(f'3 of 3: Loaded {schema}.{table} into duckdb')
    except Exception as e:
        print(f"Error while loading {schema}.{table}")
        traceback.print_exc()


1 of 3: Reading mimiciv_note.radiology_detail from parquet
2 of 3: Loading mimiciv_note.radiology_detail into duckdb
3 of 3: Loaded mimiciv_note.radiology_detail into duckdb
1 of 3: Reading mimiciv_note.discharge_detail from parquet
2 of 3: Loading mimiciv_note.discharge_detail into duckdb
3 of 3: Loaded mimiciv_note.discharge_detail into duckdb
1 of 3: Reading mimiciv_note.discharge from parquet
2 of 3: Loading mimiciv_note.discharge into duckdb
3 of 3: Loaded mimiciv_note.discharge into duckdb
1 of 3: Reading mimiciv_note.radiology from parquet
2 of 3: Loading mimiciv_note.radiology into duckdb


: 

: 